## Health/Pollution Data Treatment and Concatenation 

In [7]:
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
import os
pd.set_option('display.max_colwidth', -1)    # stop the column text truncating... 
pd.set_option("display.max_rows", None, "display.max_columns", None)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

<ipython-input-7-7cd2f3678a89>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)    # stop the column text truncating...


### 1. Load in the Datasets

In [8]:
admissions_df = pd.read_csv('../../health_data/fingertips/data_output/hospital_admissions_resp_crdv/admissions_resp_crdv_inc_strk.csv')
prevalence_df = pd.read_csv('../../health_data/fingertips/data_output/nhs_qof_dataset/nhs_qof_dataset_ccg.csv')
admissions_df = admissions_df.rename(columns={'Time period Sortable':'Year'})
prevalence_df = prevalence_df.rename(columns={'Time period Sortable':'Year'})
mortality_df = pd.read_csv('../../data_final/mortality_data_final.csv')


In [9]:
# remove the erroneous indicator 
admissions_df = admissions_df[admissions_df['Area Name']!= 'NHS Basildon And Brentwood CCG']
prevalence_df = prevalence_df[prevalence_df['Area Name']!= 'NHS Basildon And Brentwood CCG']

### 2. Rename Indicators 

In [10]:
indicator_dict = {
       'Emergency hospital admissions for COPD, all ages': 'COPD Admissions',
       'Emergency hospital admissions for respiratory disease': 'Respiratory Admissions',
       'Admissions for lower respiratory tract infections in infants aged 1 year': 'Respiratory Tract Admissions 1yr',
       'Admissions for lower respiratory tract infections in infants aged under 1 year': 'Respiratory Tract Admissions U1yr',
       'Emergency hospital admissions for asthma in adults (aged 19 years and over)': 'Asthma Admissions Over 19yr',
       'Hospital admissions for asthma (under 19 years)':'Asthma Admissions Under 19yr',
       'Emergency hospital admissions for pneumonia': 'Pneumonia Admissions',
       'CHD admissions (all ages)': 'CHD Admissions', 
       'Heart failure admissions (all ages)': 'Heart Failure Admissions',
       'Stroke admissions (Sentinel Stroke National Audit Programme)': 'Stroke Admissions',
       'Stroke admissions with history of atrial fibrillation not prescribed anticoagulation prior to stroke': 'Stroke Atrial Fib Admissions'   
}

respiratory = [
    'COPD Admissions',
    'Respiratory Admissions',
    'Respiratory Tract Admissions 1yr',
    'Respiratory Tract Admissions U1yr',
    'Asthma Admissions Over 19yr',
    'Asthma Admissions Under 19yr',
    'Pneumonia Admissions',
]
cardiov_ = [
    'CHD Admissions',
    'Heart Failure Admissions',
    'Stroke Admissions',
    'Stroke Atrial Fib Admissions' 
]

admissions_df['Indicator Name'] = admissions_df['Indicator Name'].map(indicator_dict)
admissions_df['Indicator Name'].unique()

array(['COPD Admissions', 'Respiratory Admissions',
       'Respiratory Tract Admissions 1yr',
       'Respiratory Tract Admissions U1yr', 'Asthma Admissions Over 19yr',
       'Asthma Admissions Under 19yr', 'Pneumonia Admissions',
       'CHD Admissions', 'Heart Failure Admissions', 'Stroke Admissions',
       'Stroke Atrial Fib Admissions'], dtype=object)

In [11]:
indicator_dict_q = {
       'Hypertension: QOF prevalence (all ages)': 'Hypertension Prevalence',
       'Diabetes: QOF prevalence (17+)': 'Diabetes Prevalence', 
       'CHD: QOF prevalence (all ages)': 'CHD Prevalence',
       'Heart Failure: QOF prevalence (all ages)': 'Heart Failure Prevalence' ,
       'CKD: QOF prevalence (18+)': 'CKD Prevalence', 
       'Stroke: QOF prevalence (all ages)': 'Stroke: Prevalence',
       'Atrial fibrillation: QOF prevalence': 'Atrial Fibrillation Prevalence',
       'COPD: QOF prevalence (all ages)': 'COPD Prevalence',
       'Asthma: QOF prevalence (all ages)': 'Asthma Prevalence'
}

resp_p = [
    'COPD Prevalence',
    'Asthma Prevalence'  
]

cardio_p = [
    'Hypertension Prevalence',
    'Diabetes Prevalence', 
    'CHD Prevalence',
    'Heart Failure Prevalence',
    'CKD Prevalence', 
    'Stroke: Prevalence',
    'Atrial Fibrillation Prevalence',
]

prevalence_df['Indicator Name'] = prevalence_df['Indicator Name'].map(indicator_dict_q)
prevalence_df['Indicator Name'].unique()

array(['Hypertension Prevalence', 'Diabetes Prevalence', 'CHD Prevalence',
       'Heart Failure Prevalence', 'CKD Prevalence', 'Stroke: Prevalence',
       'Atrial Fibrillation Prevalence', 'COPD Prevalence',
       'Asthma Prevalence'], dtype=object)

In [12]:
# remove locations not supported by pollution data
not_supported_poll = [
    'NHS Brent CCG',
    'NHS Croydon CCG',
    'NHS Guildford and Waverley CCG',
    'NHS Harrow CCG',
    'NHS Basildon And Brentwood CCG'
]

prevalence_df = prevalence_df[~prevalence_df['Area Name'].isin(not_supported_poll)]
admissions_df = admissions_df[~admissions_df['Area Name'].isin(not_supported_poll)]

# restrict prevalence_df and admissions_df to PERSONS only (NOT male/female, as we do not have the right indicators)
prevalence_df = prevalence_df[prevalence_df['Sex'] == 'Persons']
admissions_df = admissions_df[admissions_df['Sex'] == 'Persons']

#### Join the health data sets up (treat mortality first)

In [13]:
prevalence_df.head()

,Unnamed: 0,Unnamed: 0.1,Indicator ID,Indicator Name,Parent Code,Parent Name,Area Code,Area Name,Area Type,Sex,Age,Category Type,Category,Time period,Value,Lower CI 95.0 limit,Upper CI 95.0 limit,Lower CI 99.8 limit,Upper CI 99.8 limit,Count,Denominator,Value note,Recent Trend,Compared to England value or percentiles,Compared to percentiles,Year,New data,Compared to goal,CCG Location
3,19852,19852,219,Hypertension Prevalence,E92000001,England,E38000048,NHS Ealing CCG,CCGs (2019/20),Persons,All ages,NaN,NaN,2009/10,12.026980,11.920710,12.134068,11.859797,12.196195,42954.0,357147.0,Aggregated from all known lower geography values,NaN,Lower,Not compared,2009,NaN,NaN,Inner Radius
4,19869,19869,219,Hypertension Prevalence,E92000001,England,E38000070,NHS Hammersmith And Fulham CCG,CCGs (2019/20),Persons,All ages,NaN,NaN,2009/10,9.423858,9.290173,9.559265,9.213858,9.638136,17070.0,181136.0,Aggregated from all known lower geography values,NaN,Lower,Not compared,2009,NaN,NaN,Outer Radius
6,19877,19877,219,Hypertension Prevalence,E92000001,England,E38000079,NHS Herts Valleys CCG,CCGs (2019/20),Persons,All ages,NaN,NaN,2009/10,12.546181,12.457431,12.635472,12.406497,12.687210,66730.0,531875.0,Aggregated from all known lower geography values,NaN,Lower,Not compared,2009,NaN,NaN,Outer Radius
7,19880,19880,219,Hypertension Prevalence,E92000001,England,E38000082,NHS Hillingdon CCG,CCGs (2019/20),Persons,All ages,NaN,NaN,2009/10,12.690711,12.564328,12.818179,12.491938,12.892180,33530.0,264209.0,Aggregated from all known lower geography values,NaN,Lower,Not compared,2009,NaN,NaN,Inner Radius
8,19882,19882,219,Hypertension Prevalence,E92000001,England,E38000084,NHS Hounslow CCG,CCGs (2019/20),Persons,All ages,NaN,NaN,2009/10,11.489722,11.359422,11.621321,11.284870,11.697803,26176.0,227821.0,Aggregated from all known lower geography values,NaN,Lower,Not compared,2009,NaN,NaN,Inner Radius


In [14]:
mortality_df = mortality_df.rename(columns={'Indicator': 'Indicator Name', 'Radius': 'CCG Location', 'Mortality Rate': 'Value'})

# set the indicator type field... 
mortality_df['Indicator Type'] = 'Health - Mortality'
prevalence_df['Indicator Type'] = 'Health - Prevalence'
admissions_df['Indicator Type'] = 'Health- Admission'


In [15]:
mortality_df['Indicator Name'].unique()
mortality_df[mortality_df.duplicated()]

,Unnamed: 0,Indicator Full Name,Indicator Name,Area Name,CCG Location,Year,Time period,Value,Indicator Type


In [16]:
concat_cols = ['Year', 'Indicator Type', 'Indicator Name', 'Area Name', 'CCG Location', 'Value']


# print((mortality_df[concat_cols]).shape)
# print((prevalence_df[concat_cols]).shape)
# print((admissions_df[concat_cols]).shape)

health_df = pd.concat([
    mortality_df[concat_cols], 
    prevalence_df[concat_cols], 
    admissions_df[concat_cols]
], 
    axis=0
)


health_df.shape

(5122, 6)

In [17]:
mortality_df['Indicator Name'].unique()

array(['Under 75 Resp Disease', 'Pneumonia (all mentions)',
       'COPD (contributory cause)', '<75 Coronary Heart Disease',
       'Asthma'], dtype=object)

In [18]:
# there is an observed issus with duplicates after concatting (for the mortality frame, for now we will drop the duplicates at this point. further investigation is required.)
health_df[health_df.duplicated()].shape

(1044, 6)

In [19]:
health_df = health_df.drop_duplicates()

In [20]:
health_df[health_df.duplicated()]
health_df.shape

(4078, 6)

In [21]:
def rename_mortality_indicator(indicator_name):
    if indicator_name in ['Asthma', 'Under 75 Resp Disease', 'Pneumonia (all mentions)', 'COPD (contributory cause)', '<75 Coronary Heart Disease']:
        return indicator_name + ' Mortality Rate'
    else:
        return indicator_name

In [22]:
# final step - rename the health indicators
health_df['Indicator Name'] = health_df['Indicator Name'].apply(rename_mortality_indicator)

In [23]:
health_df['Indicator Name'].unique()

array(['Under 75 Resp Disease Mortality Rate',
       'Pneumonia (all mentions) Mortality Rate',
       'COPD (contributory cause) Mortality Rate',
       '<75 Coronary Heart Disease Mortality Rate',
       'Asthma Mortality Rate', 'Hypertension Prevalence',
       'Diabetes Prevalence', 'CHD Prevalence',
       'Heart Failure Prevalence', 'CKD Prevalence', 'Stroke: Prevalence',
       'Atrial Fibrillation Prevalence', 'COPD Prevalence',
       'Asthma Prevalence', 'COPD Admissions', 'Respiratory Admissions',
       'Respiratory Tract Admissions 1yr',
       'Respiratory Tract Admissions U1yr', 'Asthma Admissions Over 19yr',
       'Asthma Admissions Under 19yr', 'Pneumonia Admissions',
       'CHD Admissions', 'Heart Failure Admissions', 'Stroke Admissions',
       'Stroke Atrial Fib Admissions'], dtype=object)

### 3. Confirm CCG Region Location with Shape files and Get NHS CCG Distances from Heathrow Airport

The team would like to redefine the NHS CCG region location assumptions (inner/outer), intially made visually, with measured distance from heathrow in metres. 

In [24]:
# load in the shape file, with distance calculated in the geography notebook.
geog = pd.read_csv('../../health_data/geography/nhs_ccg_geometry.csv', encoding='utf-8')
geog.head(2)
# fix naming convention
geog['Area Name'] = geog['Area Name'].apply(lambda x: 'NHS Hammersmith And Fulham CCG' if x=='NHS Hammersmith and Fulham CCG' else x)

In [25]:
health_df_dist = pd.merge(health_df, geog[['Area Name', 'heathrow_distance', 'LAT', 'LONG']], how='left', on='Area Name')
# print(health_df.shape)
health_df_dist.shape

(4078, 9)

In [26]:
health_df_dist['Area Name'].unique()

array(['NHS Ealing CCG', 'NHS Hillingdon CCG', 'NHS Hounslow CCG',
       'NHS Buckinghamshire CCG', 'NHS East Berkshire CCG',
       'NHS North West Surrey CCG', 'NHS Brent CCG',
       'NHS Hammersmith And Fulham CCG', 'NHS Harrow CCG',
       'NHS Herts Valleys CCG', 'NHS Oxfordshire CCG',
       'NHS Surrey Heath CCG', 'NHS Berkshire West CCG',
       'NHS Croydon CCG', 'NHS Kingston CCG', 'NHS Merton CCG',
       'NHS Richmond CCG', 'NHS Sutton CCG', 'NHS Wandsworth CCG',
       'NHS Guildford and Waverley CCG'], dtype=object)

In [27]:
health_df_dist.shape

(4078, 9)

In [28]:
health_df_dist[health_df_dist['Area Name']== 'NHS Oxfordshire CCG'].head(100)
print(health_df_dist.shape)

duplicate = health_df_dist[health_df_dist.duplicated()] 
duplicate.shape
duplicate[duplicate['Area Name'] == 'NHS Oxfordshire CCG'].head(2)

(4078, 9)


,Year,Indicator Type,Indicator Name,Area Name,CCG Location,Value,heathrow_distance,LAT,LONG


## Treat Missing Health Data points

In [29]:
health_df_dist[health_df_dist['Value'].isna()]
#  the team will address the two points at the same time as the pollutant nans... 

,Year,Indicator Type,Indicator Name,Area Name,CCG Location,Value,heathrow_distance,LAT,LONG
2947,2015,Health- Admission,Respiratory Tract Admissions 1yr,NHS Surrey Heath CCG,Outer Radius,NaN,23497.23959,51.3281,-0.70434
2963,2016,Health- Admission,Respiratory Tract Admissions 1yr,NHS Surrey Heath CCG,Outer Radius,NaN,23497.23959,51.3281,-0.70434


There are only two missing data points, both are for the Surry Heath CCG, which is excluded from the Health vs Pollution Analysis because there is no pollution data avaialble. However, as such points may be useful for the health vs distance analysis, the team will impute with an average (as there are only 4 points in total for this indicator).

In [30]:
resp_mean_surrey = health_df_dist[(health_df_dist['Indicator Name'] == 'Respiratory Tract Admissions 1yr') & 
              (health_df_dist['Area Name'] == 'NHS Surrey Heath CCG')]['Value'].mean()

# impute for 2015
health_df_dist.loc[((health_df_dist['Year']==2015) & (health_df_dist['Area Name'] == 'NHS Surrey Heath CCG')) , 
                   ['Value']] = resp_mean_surrey

# impute for 2016
health_df_dist.loc[((health_df_dist['Year']==2016) & (health_df_dist['Area Name'] == 'NHS Surrey Heath CCG')) , 
                   ['Value']] = resp_mean_surrey

In [31]:
health_df_dist[(health_df_dist['Indicator Name'] == 'Respiratory Tract Admissions 1yr') & 
              (health_df_dist['Area Name'] == 'NHS Surrey Heath CCG')]

,Year,Indicator Type,Indicator Name,Area Name,CCG Location,Value,heathrow_distance,LAT,LONG
2947,2015,Health- Admission,Respiratory Tract Admissions 1yr,NHS Surrey Heath CCG,Outer Radius,120.84,23497.23959,51.3281,-0.70434
2963,2016,Health- Admission,Respiratory Tract Admissions 1yr,NHS Surrey Heath CCG,Outer Radius,120.84,23497.23959,51.3281,-0.70434
2979,2017,Health- Admission,Respiratory Tract Admissions 1yr,NHS Surrey Heath CCG,Outer Radius,90.01,23497.23959,51.3281,-0.70434
2995,2018,Health- Admission,Respiratory Tract Admissions 1yr,NHS Surrey Heath CCG,Outer Radius,151.67,23497.23959,51.3281,-0.70434


In [32]:
health_df_dist['CCG Location'] = health_df_dist['CCG Location'].apply(
    lambda x: x.replace('Radius', '').strip() 
)

In [33]:
# re-define the CCG locations
health_df_dist['CCG Location'] = np.where(
health_df_dist['heathrow_distance'] < 15000, 'Inner', 'Outer'
)
health_df_dist.rename(columns={'CCG Location': 'Radius Location'}, inplace=True)

### Align health with Pollution for model run

Remove our limited 2003 health indicators, as pollution starts in 2004.

In [34]:
health_df_dist = health_df_dist[health_df_dist['Year'] != 2003]

In [35]:
sorted(health_df_dist['Year'].unique())

[2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [36]:
# output to CSV 
health_df_dist.to_csv('health_data_model.csv', encoding='utf-8')

In [37]:
# Buckinghamshire, Harrow, Guildford and Waverley, Surrey Heath, Brent, Croydon


# • Buckinghamshire (excluded from pollution (no data) but keep for health analysis)
# • Harrow to be (excluded from pollution (no data) but keep for health analysis)
# • Brent (de-prioritised, only if time)  
# • Guildford/waverley - (excluded from pollution (no data) but keep for health analysis)
# • Surrey Heath CCG (excluded from pollution (no data) but keep for health analysis) 


In [38]:
# we need to create a pollution CCG mapping (as some of the CCG locations have shared pollution indicators)
pollution_map = {
    'NHS Buckinghamshire CCG': 'No Pollution',
    'NHS Ealing CCG': 'Ealing', 
    'NHS East Berkshire CCG': 'East Berkshire', 
    'NHS Hillingdon CCG': 'Hillingdon',
    'NHS Hounslow CCG': 'Hounslow', 
    'NHS North West Surrey CCG': 'Surrey Heartlands',
    'NHS Berkshire West CCG': 'Reading', 
    'NHS Brent CCG': 'No Pollution',
    'NHS Croydon CCG': 'No Pollution',
    'NHS Guildford and Waverley CCG': 'No Pollution',
    'NHS Hammersmith And Fulham CCG': 'Hammersmith and Fulham', 
    'NHS Harrow CCG': 'No Pollution',
    'NHS Herts Valleys CCG': 'Watford', 
    'NHS Kingston CCG': 'South West London', 
    'NHS Merton CCG': 'South West London', 
    'NHS Oxfordshire CCG': 'Oxford', 
    'NHS Richmond CCG': 'South West London', 
    'NHS Surrey Heath CCG': 'No Pollution',
    'NHS Sutton CCG': 'South West London', 
    'NHS Wandsworth CCG': 'South West London'
}


health_df_dist['CCG_Poll_Map'] = health_df_dist['Area Name'].map(pollution_map)


In [39]:
# health_df_dist['CCG_Poll_Map'].unique()

### 4. Transform Health DF into Multi-Variate format

In [40]:
# restrict the columns to those requried 
health_df_dist_wide = health_df_dist.copy()
health_df_dist_wide = health_df_dist_wide[['Year', 'Area Name','Indicator Name', 'Value', 'Radius Location', 'heathrow_distance', 'CCG_Poll_Map', 'LAT', 'LONG']]

health_df_pivot = health_df_dist_wide.pivot_table(
    values='Value',
    index= ['Year', 'Area Name', 'Radius Location', 'heathrow_distance', 'CCG_Poll_Map', 'LAT', 'LONG'],
    columns='Indicator Name'
).reset_index().rename(columns={'Indicator Name': 'Index'})

# output to CSV (blanked out for now)
health_df_pivot.to_csv('health_data_model_wide.csv', encoding='utf-8')

### 5. Combine with Pollution Data 

In [41]:
# this file is not grouped 1:1 (there are duplicate years, use final file instead)
outer_poll = pd.read_csv('../../data_final/yearlyOuterCCGgrouped.csv', encoding='utf-8')

inner_poll = pd.read_csv('../../pollution_data/inner_ccg_agg.csv', encoding='utf-8')

In [42]:
outer_poll[outer_poll['CCG']=='Oxford'].head(50)

,Date,CCG,Nitrogen dioxide,Ozone,PM10 particulate matter (Hourly measured),PM2.5 particulate matter (Hourly measured),Sulphur dioxide
1,2004-12-31,Oxford,63.545897,40.376279,27.789444,NaN,NaN
4,2005-12-31,Oxford,49.362863,39.652825,25.594434,NaN,NaN
7,2006-12-31,Oxford,55.820565,45.274464,28.368755,NaN,NaN
11,2007-12-31,Oxford,58.257026,39.945394,25.961705,NaN,NaN
15,2008-12-31,Oxford,54.460016,41.216238,24.872508,NaN,NaN
19,2009-12-31,Oxford,54.833902,46.349644,25.106133,NaN,NaN
22,2010-12-31,Oxford,60.259746,49.677520,23.630868,NaN,NaN
26,2011-12-31,Oxford,52.553017,38.608096,22.913411,NaN,NaN
30,2012-12-31,Oxford,57.906920,35.100020,20.511619,NaN,NaN
34,2013-12-31,Oxford,52.145280,36.761817,22.290244,NaN,NaN


In [43]:
# return only the year from the full date string
inner_poll['Date'] = inner_poll['Date'].apply(lambda x: x[:4])
outer_poll['Date'] = outer_poll['Date'].apply(lambda x: x[:4])

# rename columns to match health data 
inner_poll.rename(columns={'CCG':'CCG_Poll_Map', 'Date':'Year'}, inplace=1)
outer_poll.rename(columns={'CCG':'CCG_Poll_Map', 'Date':'Year'}, inplace=1)

# append reading ccg name for some values 
outer_poll['CCG_Poll_Map']  = outer_poll['CCG_Poll_Map'].apply(lambda x: 'Reading' if 'Reading' in x else x)

# fix the year column
inner_poll['Year'] =  pd.to_datetime(
    (inner_poll['Year']), 
    format='%Y').dt.year

# fix the year column
outer_poll['Year'] =  pd.to_datetime(
    (outer_poll['Year']), 
    format='%Y').dt.year


In [44]:
# pivot the table so it is the same format as health and outer pollution
inner_poll_p = inner_poll[['CCG_Poll_Map', 'Year', 'Pollutant', 'Indicator Value (R µg/m3)']].pivot_table(
    index = ['CCG_Poll_Map', 'Year',],
    columns= 'Pollutant',
    values = 'Indicator Value (R µg/m3)'
).reset_index()

In [45]:
inner_poll_p.head(5)

Pollutant,CCG_Poll_Map,Year,Carbon monoxide,Nitric Oxide,Nitrogen dioxide,Oxides of Nitrogen,Ozone,PM10 Particulate matter,PM10 particulate matter (Hourly measured),PM2.5 particulate matter (Hourly measured),Sulphur dioxide
0,Ealing,2004,0.770049,NaN,57.882938,NaN,33.992080,NaN,23.333576,15.391342,10.043232
1,Ealing,2005,0.762150,NaN,52.862974,NaN,28.594453,NaN,25.916647,21.009842,8.370425
2,Ealing,2006,0.730095,NaN,55.035635,NaN,32.527539,NaN,25.443425,18.450045,6.206311
3,Ealing,2007,0.651978,NaN,54.247691,NaN,28.473891,NaN,24.401769,NaN,6.530478
4,Ealing,2008,0.502517,NaN,58.848371,NaN,29.085305,NaN,22.141475,11.795321,5.656652


we will drop the indicators that are present in closer regions and not present in further away regions (as we need the same indicators bewteen both regions for comparisons) 

In [46]:
set(inner_poll_p.columns.tolist()) - set(outer_poll.columns.tolist())

{'Carbon monoxide',
 'Nitric Oxide',
 'Oxides of Nitrogen',
 'PM10 Particulate matter'}

In [47]:
inner_poll_p.drop(columns=['Carbon monoxide',
 'Nitric Oxide',
 'Oxides of Nitrogen',
 'PM10 Particulate matter',], inplace=True)

In [48]:
inner_poll_p.head()

Pollutant,CCG_Poll_Map,Year,Nitrogen dioxide,Ozone,PM10 particulate matter (Hourly measured),PM2.5 particulate matter (Hourly measured),Sulphur dioxide
0,Ealing,2004,57.882938,33.992080,23.333576,15.391342,10.043232
1,Ealing,2005,52.862974,28.594453,25.916647,21.009842,8.370425
2,Ealing,2006,55.035635,32.527539,25.443425,18.450045,6.206311
3,Ealing,2007,54.247691,28.473891,24.401769,NaN,6.530478
4,Ealing,2008,58.848371,29.085305,22.141475,11.795321,5.656652


concat the inner/outer pollution datasets

In [49]:
poll_df = pd.concat([inner_poll_p, outer_poll], axis=0)

In [50]:
poll_df.head(1)

,CCG_Poll_Map,Year,Nitrogen dioxide,Ozone,PM10 particulate matter (Hourly measured),PM2.5 particulate matter (Hourly measured),Sulphur dioxide
0,Ealing,2004,57.882938,33.99208,23.333576,15.391342,10.043232


In [51]:
print(inner_poll_p.shape)
print(outer_poll.shape)

print(poll_df.shape)
poll_df.isna().sum()

(102, 7)
(65, 7)
(167, 7)


CCG_Poll_Map                                  0  
Year                                          0  
Nitrogen dioxide                              0  
Ozone                                         97 
PM10 particulate matter (Hourly measured)     18 
PM2.5 particulate matter (Hourly measured)    92 
Sulphur dioxide                               133
dtype: int64

### Pollution Data Treatment (assess the NaNs here before merging with Health)

In [52]:
outer_poll[outer_poll['CCG_Poll_Map'] == 'Oxford'].head(50)

,Year,CCG_Poll_Map,Nitrogen dioxide,Ozone,PM10 particulate matter (Hourly measured),PM2.5 particulate matter (Hourly measured),Sulphur dioxide
1,2004,Oxford,63.545897,40.376279,27.789444,NaN,NaN
4,2005,Oxford,49.362863,39.652825,25.594434,NaN,NaN
7,2006,Oxford,55.820565,45.274464,28.368755,NaN,NaN
11,2007,Oxford,58.257026,39.945394,25.961705,NaN,NaN
15,2008,Oxford,54.460016,41.216238,24.872508,NaN,NaN
19,2009,Oxford,54.833902,46.349644,25.106133,NaN,NaN
22,2010,Oxford,60.259746,49.677520,23.630868,NaN,NaN
26,2011,Oxford,52.553017,38.608096,22.913411,NaN,NaN
30,2012,Oxford,57.906920,35.100020,20.511619,NaN,NaN
34,2013,Oxford,52.145280,36.761817,22.290244,NaN,NaN


In [53]:
poll_df.isna().sum()

CCG_Poll_Map                                  0  
Year                                          0  
Nitrogen dioxide                              0  
Ozone                                         97 
PM10 particulate matter (Hourly measured)     18 
PM2.5 particulate matter (Hourly measured)    92 
Sulphur dioxide                               133
dtype: int64

**Finding 1 - Sulphur Dioxide: we can say with certainty, that for the model analysis, Sulphur Dioxide should be dropped, as the majority of points is missing (133/165)** 

2: check ozone, 58% is missing, but over what periods or areas? 

In [54]:
# 2: check ozone, 58% is missing, but over what periods or areas? 
print(97/167)
print(13/17)
tmp = poll_df[['Year', 'CCG_Poll_Map', 'Ozone',]]

na_tmp = tmp[tmp['Ozone'].isnull()]
print('number of years is ', len(list(tmp['Year'].unique())))
print('number of CCGs is ', len(list(tmp['CCG_Poll_Map'].unique())))
print(na_tmp['Year'].value_counts())
print(na_tmp['CCG_Poll_Map'].value_counts())

0.5808383233532934
0.7647058823529411
number of years is  17
number of CCGs is  10
2018    7
2017    7
2020    6
2019    6
2008    6
2009    6
2011    6
2012    6
2013    6
2014    6
2015    6
2016    6
2010    5
2007    5
2006    5
2005    4
2004    4
Name: Year, dtype: int64
Hillingdon                17
Surrey Heartlands         17
East Berkshire            17
Reading                   15
Hammersmith and Fulham    14
Watford                   13
Ealing                    4 
Name: CCG_Poll_Map, dtype: int64


**Observation two: Ozone is 56% missing, in 7/10 different Areas (and over 75% missing in 6/10 Areas). This gap (and its nature, missing across most data points for 6/10) is too large to fill. From 2005 onwards the data is missing in over 50% of CCG regions. We will subsequently drop Ozone from the model analysis.**

3: check PM2.5

In [55]:
poll_df.head(1)

,CCG_Poll_Map,Year,Nitrogen dioxide,Ozone,PM10 particulate matter (Hourly measured),PM2.5 particulate matter (Hourly measured),Sulphur dioxide
0,Ealing,2004,57.882938,33.99208,23.333576,15.391342,10.043232


Finally, rename the CCGs to match the health indicators

In [56]:
# shorten pollution indicator names
poll_df.rename(columns= {
    'PM10 particulate matter (Hourly measured)': 'PM10',
    'PM2.5 particulate matter (Hourly measured)': 'PM2.5',
}, inplace=True)


In [57]:
# 3: check PM2.5
print(92/167)
tmp = poll_df[['Year', 'CCG_Poll_Map', 'PM2.5',]]

# na_tmp = tmp[(tmp['PM2.5'].isnull()) & (tmp['Year'] > 2010)]
na_tmp = tmp[(tmp['PM2.5'].isnull())]
# na_tmp.head(50)

print('number of years is ', len(list(tmp['Year'].unique())))
print('number of CCGs is ', len(list(tmp['CCG_Poll_Map'].unique())))
print(na_tmp['Year'].value_counts())
print(na_tmp['CCG_Poll_Map'].value_counts())

0.5508982035928144
number of years is  17
number of CCGs is  10
2007    7
2004    6
2018    6
2017    6
2015    6
2014    6
2006    6
2009    5
2005    5
2008    5
2020    5
2011    5
2019    5
2013    5
2016    5
2012    5
2010    4
Name: Year, dtype: int64
Hounslow                  17
Oxford                    17
Reading                   15
Hammersmith and Fulham    14
Watford                   12
South West London         11
Ealing                    5 
East Berkshire            1 
Name: CCG_Poll_Map, dtype: int64


**Observation 3: Similarly to Ozone, PM2.5 is missing from 55% of the Pollution dataset. From 2005 onwards, data is missing in 5 or more regions (50%), which is unacceptable. Therefore we will also remove PM2.5 from the model analysis.**

4: check PM10 

In [58]:
# 3: check PM10 
print(18/167)
tmp = poll_df[['Year', 'CCG_Poll_Map', 'PM10',]]

# na_tmp = tmp[(tmp['PM2.5'].isnull()) & (tmp['Year'] > 2010)]
na_tmp = tmp[(tmp['PM10'].isnull())]
# na_tmp.head(50)

print('number of years is ', len(list(tmp['Year'].unique())))
print('number of CCGs is ', len(list(tmp['CCG_Poll_Map'].unique())))
print(na_tmp['Year'].value_counts())
print(na_tmp['CCG_Poll_Map'].value_counts())

0.10778443113772455
number of years is  17
number of CCGs is  10
2006    2
2015    1
2017    1
2018    1
2019    1
2020    1
2004    1
2005    1
2007    1
2014    1
2008    1
2009    1
2010    1
2011    1
2012    1
2013    1
2016    1
Name: Year, dtype: int64
Hounslow    17
Reading     1 
Name: CCG_Poll_Map, dtype: int64


A different story is presented for PM10... 
PM10 is only missing from two Areas... Reading (2006) and Hounslow (across the board). Given the team is trying to discover a relationship between pollutants and health ailments at specific locations it may be more appropriate to exclude Hounslow from the PM10 analysis than to try and impute the complete set of results. For Reading (2006), the team will impute the mean value, as there is only one missing data point.  

In [59]:
poll_df[poll_df['CCG_Poll_Map']=='Hammersmith and Fulham'].head(100)

,CCG_Poll_Map,Year,Nitrogen dioxide,Ozone,PM10,PM2.5,Sulphur dioxide
0,Hammersmith and Fulham,2004,58.655707,NaN,26.158435,NaN,9.171105
3,Hammersmith and Fulham,2005,52.325337,NaN,30.948655,NaN,8.271812
6,Hammersmith and Fulham,2006,61.199160,NaN,26.324390,NaN,6.046027
10,Hammersmith and Fulham,2007,60.160685,NaN,25.631845,NaN,4.742114
14,Hammersmith and Fulham,2008,56.576836,NaN,25.863772,NaN,NaN
18,Hammersmith and Fulham,2009,50.337135,NaN,20.222782,NaN,NaN
25,Hammersmith and Fulham,2011,83.648985,NaN,32.772381,NaN,NaN
29,Hammersmith and Fulham,2012,92.041562,NaN,37.709165,NaN,NaN
33,Hammersmith and Fulham,2013,77.797963,NaN,26.497921,NaN,NaN
37,Hammersmith and Fulham,2014,80.758056,NaN,28.406975,NaN,NaN


In [60]:
# impute single missing value with the mean
poll_df[poll_df['CCG_Poll_Map'] == 'Reading']['PM10']
rdg_pm10_mean = poll_df[poll_df['CCG_Poll_Map'] == 'Reading']['PM10'].mean()
rdg_n2o_mean = poll_df[poll_df['CCG_Poll_Map'] == 'Reading']['Nitrogen dioxide'].mean()

hammer_pm10_mean = poll_df[poll_df['CCG_Poll_Map'] == 'Hammersmith and Fulham']['PM10'].mean()
hammer_n20_mean = poll_df[poll_df['CCG_Poll_Map'] == 'Hammersmith and Fulham']['Nitrogen dioxide'].mean()


Final observation:  there are three rows missing from the pollution: Reading (2004/2005) & Hammersmith and Fulham (2010). In order to match the health indicators, I recommend the team imputes an average for these years.

In [61]:
points_to_add = pd.DataFrame([
    {
        'CCG_Poll_Map': 'Reading',
        'Year':2004 ,
        'Nitrogen dioxide': rdg_n2o_mean,
        'PM10': rdg_pm10_mean,
    },
    
    {
        'CCG_Poll_Map': 'Reading',
        'Year':2005,
        'Nitrogen dioxide': rdg_n2o_mean,
        'PM10' :rdg_pm10_mean,
    },
    {
        'CCG_Poll_Map': 'Hammersmith and Fulham',
        'Year':2010 ,
        'Nitrogen dioxide': hammer_n20_mean,
        'PM10': hammer_pm10_mean,
    },
])



points_to_add.head()

,CCG_Poll_Map,Year,Nitrogen dioxide,PM10
0,Reading,2004,40.215141,24.900648
1,Reading,2005,40.215141,24.900648
2,Hammersmith and Fulham,2010,66.824001,26.862585


In [62]:
print(poll_df.shape)

poll_df = pd.concat([poll_df, points_to_add], axis=0, ignore_index=True)

print(poll_df.shape)

(167, 7)
(170, 7)


In [63]:
poll_df[poll_df['CCG_Poll_Map'] == 'Reading']

,CCG_Poll_Map,Year,Nitrogen dioxide,Ozone,PM10,PM2.5,Sulphur dioxide
110,Reading,2006,39.607957,NaN,NaN,NaN,NaN
114,Reading,2007,53.170798,NaN,31.676369,NaN,NaN
118,Reading,2008,43.441850,NaN,24.531073,NaN,NaN
122,Reading,2009,44.314326,NaN,24.829365,NaN,NaN
125,Reading,2010,46.989329,NaN,25.552918,NaN,NaN
129,Reading,2011,45.956385,NaN,29.421368,NaN,NaN
133,Reading,2012,46.206437,NaN,28.283750,NaN,NaN
137,Reading,2013,47.014253,NaN,23.624317,NaN,NaN
141,Reading,2014,41.870020,NaN,28.024767,NaN,NaN
145,Reading,2015,38.862140,NaN,25.260289,NaN,NaN


In [64]:
# remove the pollution indicators that have too much data to fill
poll_df = poll_df.drop(columns=['Sulphur dioxide', 'PM2.5', 'Ozone'])

In [65]:
# Fill in missing value with mean using Reading LOC.
poll_df.loc[((poll_df['Year']==2006) & (poll_df['CCG_Poll_Map'] == 'Reading')) , ['PM10']] = rdg_pm10_mean

In [66]:
poll_df[poll_df['CCG_Poll_Map']=='Reading'].head(10)

,CCG_Poll_Map,Year,Nitrogen dioxide,PM10
110,Reading,2006,39.607957,24.900648
114,Reading,2007,53.170798,31.676369
118,Reading,2008,43.441850,24.531073
122,Reading,2009,44.314326,24.829365
125,Reading,2010,46.989329,25.552918
129,Reading,2011,45.956385,29.421368
133,Reading,2012,46.206437,28.283750
137,Reading,2013,47.014253,23.624317
141,Reading,2014,41.870020,28.024767
145,Reading,2015,38.862140,25.260289


In [67]:
poll_df_long = poll_df.melt(
    id_vars = ['Year', 'CCG_Poll_Map'],
    var_name = 'Indicator Name',
    value_name = 'Value',
)

In [68]:
print(poll_df.shape)
poll_df.isna().sum()

(170, 4)


CCG_Poll_Map        0 
Year                0 
Nitrogen dioxide    0 
PM10                17
dtype: int64

In [69]:
poll_df.head()

,CCG_Poll_Map,Year,Nitrogen dioxide,PM10
0,Ealing,2004,57.882938,23.333576
1,Ealing,2005,52.862974,25.916647
2,Ealing,2006,55.035635,25.443425
3,Ealing,2007,54.247691,24.401769
4,Ealing,2008,58.848371,22.141475


In [70]:
# output pollution file to CSV
poll_df.to_csv('pollution_ccg_model_wide.csv', encoding='utf-8')

poll_df_long.to_csv('pollution_ccg_model.csv', encoding='utf-8')


## merge pollution and health data


In [71]:
health_df_pivot[health_df_pivot['CCG_Poll_Map'] != 'No Pollution'].groupby('Year').agg({'Area Name': 'nunique'})

,Area Name
Year,
2004,14
2005,14
2006,14
2007,14
2008,14
2009,14
2010,14
2011,14
2012,14


As expected, there are 14 CCGs which should have a pollution reading (double check final files after merge). We should now remove the CCGs which have no pollution points


In [72]:
## at this point, remove the CCGs where there is no pollution recorded
health_df_pivot = health_df_pivot[health_df_pivot['CCG_Poll_Map'] != 'No Pollution']

In [73]:
health_df_pivot['CCG_Poll_Map'].unique()

array(['Reading', 'Ealing', 'East Berkshire', 'Hammersmith and Fulham',
       'Watford', 'Hillingdon', 'Hounslow', 'South West London',
       'Surrey Heartlands', 'Oxford'], dtype=object)

In [74]:
# poll_df.head()
# poll_df['year_ccg_poll_Map'] = poll_df.apply(lambda x: (str(x['Year']) + '_' + x['CCG_Poll_Map'].lower()).strip(), axis=1)

In [75]:
health_poll_df = pd.merge(health_df_pivot, poll_df, how='inner', on=['Year', 'CCG_Poll_Map'])

In [76]:
health_poll_df.groupby('Year').agg({'Area Name': 'nunique'})

,Area Name
Year,
2004,14
2005,14
2006,14
2007,14
2008,14
2009,14
2010,14
2011,14
2012,14


In [77]:
poll_df[poll_df['CCG_Poll_Map'] == 'Reading']

,CCG_Poll_Map,Year,Nitrogen dioxide,PM10
110,Reading,2006,39.607957,24.900648
114,Reading,2007,53.170798,31.676369
118,Reading,2008,43.441850,24.531073
122,Reading,2009,44.314326,24.829365
125,Reading,2010,46.989329,25.552918
129,Reading,2011,45.956385,29.421368
133,Reading,2012,46.206437,28.283750
137,Reading,2013,47.014253,23.624317
141,Reading,2014,41.870020,28.024767
145,Reading,2015,38.862140,25.260289


In [78]:
# output datasets to CSV in WIDE and long format.
# (also remember to include the INdicator TYPE (get manually from the column names))
health_poll_df_long = health_poll_df.melt(
    id_vars = ['Year', 'Area Name', 'heathrow_distance', 'CCG_Poll_Map', 'Radius Location', 'LAT', 'LONG'],
    var_name = 'Indicator Name',
    value_name = 'Value',
)

In [79]:
health_poll_df_long['Indicator Name'].unique()

array(['<75 Coronary Heart Disease Mortality Rate',
       'Asthma Admissions Over 19yr', 'Asthma Admissions Under 19yr',
       'Asthma Mortality Rate', 'Asthma Prevalence',
       'Atrial Fibrillation Prevalence', 'CHD Admissions',
       'CHD Prevalence', 'CKD Prevalence',
       'COPD (contributory cause) Mortality Rate', 'COPD Admissions',
       'COPD Prevalence', 'Diabetes Prevalence',
       'Heart Failure Admissions', 'Heart Failure Prevalence',
       'Hypertension Prevalence',
       'Pneumonia (all mentions) Mortality Rate', 'Pneumonia Admissions',
       'Respiratory Admissions', 'Respiratory Tract Admissions 1yr',
       'Respiratory Tract Admissions U1yr', 'Stroke Admissions',
       'Stroke Atrial Fib Admissions', 'Stroke: Prevalence',
       'Under 75 Resp Disease Mortality Rate', 'Nitrogen dioxide', 'PM10'],
      dtype=object)

In [80]:
print([x for x in health_poll_df_long['Indicator Name'].unique() if 'admission' in x.lower()])
print('\n')
print([x for x in health_poll_df_long['Indicator Name'].unique() if 'prevalence' in x.lower()])


admissions = ['Asthma Admissions Over 19yr', 'Asthma Admissions Under 19yr', 'CHD Admissions', 'COPD Admissions', 'Heart Failure Admissions', 'Pneumonia Admissions', 'Respiratory Admissions', 'Respiratory Tract Admissions 1yr', 'Respiratory Tract Admissions U1yr', 'Stroke Admissions', 'Stroke Atrial Fib Admissions']
prev = ['Asthma Prevalence', 'Atrial Fibrillation Prevalence', 'CHD Prevalence', 'CKD Prevalence', 'COPD Prevalence', 'Diabetes Prevalence', 'Heart Failure Prevalence', 'Hypertension Prevalence', 'Stroke: Prevalence']
pollution = ['Nitrogen dioxide', 'Ozone', 'PM10', 'PM2.5', 'Sulphur dioxide']


health_poll_df_long['Indicator Type'] = np.where(
health_poll_df_long['Indicator Name'].isin(admissions), 'Health - Admissions',
    np.where(health_poll_df_long['Indicator Name'].isin(prev), 'Health - Prevalence',    
             (np.where(health_poll_df_long['Indicator Name'].isin(pollution), 'Air Pollutant', 'Health - Mortality'))))



['Asthma Admissions Over 19yr', 'Asthma Admissions Under 19yr', 'CHD Admissions', 'COPD Admissions', 'Heart Failure Admissions', 'Pneumonia Admissions', 'Respiratory Admissions', 'Respiratory Tract Admissions 1yr', 'Respiratory Tract Admissions U1yr', 'Stroke Admissions', 'Stroke Atrial Fib Admissions']


['Asthma Prevalence', 'Atrial Fibrillation Prevalence', 'CHD Prevalence', 'CKD Prevalence', 'COPD Prevalence', 'Diabetes Prevalence', 'Heart Failure Prevalence', 'Hypertension Prevalence', 'Stroke: Prevalence']


In [81]:
health_poll_df_long.head(5)

,Year,Area Name,heathrow_distance,CCG_Poll_Map,Radius Location,LAT,LONG,Indicator Name,Value,Indicator Type
0,2004,NHS Berkshire West CCG,49791.74229,Reading,Outer,51.4458,-1.16963,<75 Coronary Heart Disease Mortality Rate,55.28,Health - Mortality
1,2004,NHS Ealing CCG,11465.53093,Ealing,Inner,51.5244,-0.31405,<75 Coronary Heart Disease Mortality Rate,70.77,Health - Mortality
2,2004,NHS East Berkshire CCG,15803.71343,East Berkshire,Outer,51.4541,-0.68026,<75 Coronary Heart Disease Mortality Rate,62.29,Health - Mortality
3,2004,NHS Hammersmith And Fulham CCG,16573.14549,Hammersmith and Fulham,Outer,51.4873,-0.21735,<75 Coronary Heart Disease Mortality Rate,65.68,Health - Mortality
4,2004,NHS Herts Valleys CCG,29114.52213,Watford,Outer,51.7317,-0.45521,<75 Coronary Heart Disease Mortality Rate,48.24,Health - Mortality


In [82]:
#check that all indicators have been classified
mo = health_poll_df_long[health_poll_df_long['Indicator Type'] == 'Health - Mortality']['Indicator Name'].unique()
pv = health_poll_df_long[health_poll_df_long['Indicator Type'] == 'Health - Prevalence']['Indicator Name'].unique()
ad = health_poll_df_long[health_poll_df_long['Indicator Type'] == 'Health - Admissions']['Indicator Name'].unique()
po = health_poll_df_long[health_poll_df_long['Indicator Type'] == 'Air Pollutant']['Indicator Name'].unique()

# empty set means well done
print(set(list(health_poll_df_long['Indicator Name'])) - set(list(mo) + list(pv) + list(ad) + list(po)))

set()


drop duplicates for the long format data (need to review why duplicates are created, but for now, first will be retained, which is great, because we need one row per year) 

In [83]:
health_poll_df_long = health_poll_df_long.drop_duplicates()
health_poll_df_long[health_poll_df_long.duplicated()].head(10)

,Year,Area Name,heathrow_distance,CCG_Poll_Map,Radius Location,LAT,LONG,Indicator Name,Value,Indicator Type


In [84]:
# output datasets to CSV in WIDE and long format.
# (also remember to include the INdicator TYPE (get manually from the column names))
health_poll_df.to_csv('health_pollution_model_wide.csv', encoding='utf-8')
health_poll_df_long.to_csv('health_pollution_model.csv', encoding='utf-8')

### 5A Data Sanity Check

In [85]:
test_1 = pd.read_csv('health_pollution_model_wide.csv', encoding='utf-8')

In [86]:
test_1[['Nitrogen dioxide', 'PM10']].isna().sum()

Nitrogen dioxide    0 
PM10                16
dtype: int64

In [87]:
a = test_1.groupby('Year').agg({'Area Name': 'nunique'})

In [88]:
# Q1, what CCGs are missing in 2019/2020 and why?

ccgs_2017 = list(test_1[test_1['Year']==2017]['Area Name'].unique())
ccgs_2019 = list(test_1[test_1['Year']==2019]['Area Name'].unique())
ccgs_2018 = list(test_1[test_1['Year']==2018]['Area Name'].unique())

print('missing cgs for 2019 are, ',  (set(ccgs_2017) - set(ccgs_2019)))
print('missing cgs for 2018 are, ',  (set(ccgs_2017) - set(ccgs_2018)))

missing cgs for 2019 are,  set()
missing cgs for 2018 are,  set()


In [89]:
# are the missing CCGs im the health dataset alone ? If so, we would infer an error when joining the pollution data
test_2 = pd.read_csv('health_data_model_wide.csv', encoding='utf-8')
test_2.head()

,Unnamed: 0,Year,Area Name,Radius Location,heathrow_distance,CCG_Poll_Map,LAT,LONG,<75 Coronary Heart Disease Mortality Rate,Asthma Admissions Over 19yr,Asthma Admissions Under 19yr,Asthma Mortality Rate,Asthma Prevalence,Atrial Fibrillation Prevalence,CHD Admissions,CHD Prevalence,CKD Prevalence,COPD (contributory cause) Mortality Rate,COPD Admissions,COPD Prevalence,Diabetes Prevalence,Heart Failure Admissions,Heart Failure Prevalence,Hypertension Prevalence,Pneumonia (all mentions) Mortality Rate,Pneumonia Admissions,Respiratory Admissions,Respiratory Tract Admissions 1yr,Respiratory Tract Admissions U1yr,Stroke Admissions,Stroke Atrial Fib Admissions,Stroke: Prevalence,Under 75 Resp Disease Mortality Rate
0,0,2004,NHS Berkshire West CCG,Outer,49791.74229,Reading,51.4458,-1.16963,55.28,NaN,NaN,NaN,NaN,NaN,552.864,NaN,NaN,NaN,NaN,NaN,NaN,121.822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2004,NHS Brent CCG,Outer,16247.54245,No Pollution,51.5644,-0.27568,68.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2004,NHS Buckinghamshire CCG,Outer,38818.06153,No Pollution,51.7414,-0.80644,46.66,NaN,NaN,NaN,NaN,NaN,837.763,NaN,NaN,NaN,NaN,NaN,NaN,109.875,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2004,NHS Croydon CCG,Outer,28645.84070,No Pollution,51.3660,-0.07761,59.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2004,NHS Ealing CCG,Inner,11465.53093,Ealing,51.5244,-0.31405,70.77,NaN,NaN,NaN,NaN,NaN,1015.463,NaN,NaN,NaN,NaN,NaN,NaN,199.538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
# Q1- testing with health, what CCGs are missing in 2019/2020 and why?

ccgs_2017 = list(test_2[test_2['Year']==2017]['Area Name'].unique())
ccgs_2019 = list(test_2[test_2['Year']==2019]['Area Name'].unique())
ccgs_2018 = list(test_2[test_2['Year']==2018]['Area Name'].unique())

print('missing cgs for 2019 are, ',  (set(ccgs_2017) - set(ccgs_2019)))
print('missing cgs for 2018 are, ',  (set(ccgs_2017) - set(ccgs_2018)))

missing cgs for 2019 are,  {'NHS Brent CCG', 'NHS Croydon CCG', 'NHS Harrow CCG', 'NHS Guildford and Waverley CCG'}
missing cgs for 2018 are,  {'NHS Brent CCG', 'NHS Croydon CCG', 'NHS Harrow CCG', 'NHS Guildford and Waverley CCG'}


The same ccgs appear to be missing for the health datasets, so no errors appear to have been made in concatenation.
To test this once and for all, lets look at the original imported health datasets.
Also, as the ccgs are missing across the board, this will not affect model inputs (who will need an equal x/y axis). So given this, lets move on! 

The CCGs have sincee been removed for the health/pollution data (as they do not have pollution readings)

In [91]:
health_df.head()
print(sorted(health_df[health_df['Area Name'] == 'NHS Harrow CCG'].Year.unique())) 
print(sorted(health_df[health_df['Area Name'] == 'NHS Brent CCG'].Year.unique())) 
print(sorted(health_df[health_df['Area Name'] == 'NHS Guildford and Waverley CCG'].Year.unique())) 
print(sorted(health_df[health_df['Area Name'] == 'NHS Croydon CCG'].Year.unique())) 
print('\n')
print('you can see that the four regions do not have any data avaialble post 2017, after further analysis they are only available for mortality inidicators.')


[2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
[2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
[2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
[2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]


you can see that the four regions do not have any data avaialble post 2017, after further analysis they are only available for mortality inidicators.


### 6. Run Multivariate EDA analysis 

In [92]:
# # lets make a correlation heatmap matrix!
# sns.set(rc={'figure.figsize':(12,9)})

# # https://seaborn.pydata.org/generated/seaborn.diverging_palette.html
# cmap= sns.diverging_palette(240, 10, as_cmap=True)

# admission_corr = health_poll_df_long.corr()
# sns.heatmap(admission_corr, cmap=cmap)

# plt.title('Correlation Heatmap of Hospital Admissions and Air Pollution Indicators \n on Selected Areas Around Heathrow Airport')
# # plt.show()

Observations of interest:

* Various hospital admissions are negatively correlated with heathrow distance, supporting our theory that residents living closer to heathrow experience more health ailments
* Observed pollutants appear to be (weakly) positively correlated with distance, further investigation will be required
* Nitrogen Dioxide is positively correlated with 6 hospital admissions
* PM10 particulate matter is positively correlated with with 4 hospital admissions

In [93]:
# # lets make a correlation heatmap matrix!
# sns.set(rc={'figure.figsize':(12,9)})

# # https://seaborn.pydata.org/generated/seaborn.diverging_palette.html
# # cmap = sns.diverging_palette(220, 20, as_cmap=True)
# cmap= sns.diverging_palette(240, 10, as_cmap=True)

# prev_corr = prevalence_df_poll.corr()
# sns.heatmap(prev_corr, cmap=cmap)

# plt.title('Correlation Heatmap of Disease Prevalence and Air Pollution Indicators \n on Selected Areas Around Heathrow Airport')
# plt.show()

Observations of interest:
* Distance from the airport is positively corrleated with most disease prevalence indicators), which contradicts our theory that that residents living closer to heathrow experience more health ailments we would have expected the opposite
* PM10 particulate matter is positively correlated with Asthma, Hypertension 
* Ozone is positively correlated with COPD prevalence (Further investigation required)
    

We will use pair plots to give an alternative visualisation

In [94]:
# g = sns.pairplot(
#     data=prevalence_df_poll,
#     x_vars=['Year', 'heathrow_distance','Nitrogen dioxide', 'Ozone', 'PM10', 'PM2.5', 'Sulphur dioxide'],
#     y_vars=['Year','heathrow_distance', 'Asthma Prevalence', 'Atrial Fibrillation Prevalence', 'CHD Prevalence', 'CKD Prevalence', 'COPD Prevalence', 'Diabetes Prevalence', 'Heart Failure Prevalence', 'Hypertension Prevalence', 'Stroke: Prevalence',]
# )

# g.fig.suptitle("Multivariate Correlation Plots for London Disease Prevalence and Air Pollution (2009-2019)", y=1.02, size=16) # y= some height>1

# plt.show()

In [95]:
# sns.pairplot(
#     data=admission_df_poll,
#     x_vars=['Year', 'heathrow_distance','Nitrogen dioxide', 'Ozone', 'PM10', 'PM2.5', 'Sulphur dioxide'],
#     y_vars=['Year','heathrow_distance', 'Asthma Admissions Over 19yr', 'Asthma Admissions Under 19yr', 'CHD Admissions', 'COPD Admissions', 'Heart Failure Admissions', 'Pneumonia Admissions', 'Respiratory Admissions', 'Respiratory Tract Admissions 1yr', 'Respiratory Tract Admissions U1yr', 'Stroke Admissions', 'Stroke Atrial Fib Admissions',],
#     height=3, 
# #     aspect=0.8
# )
    
    
# g.fig.suptitle("Multivariate Correlation Plots for London Hospital Admissions and Air Pollution (2009-2019/2013-2018)", y=1.02, size=16) # y= some height>1
# plt.tight_layout()
# plt.show()

### Export file (Used for Front End and Model)

In [96]:
# geog.head()

In [97]:
# # get lat long (for front end design)
# admission_df_poll_latlong = pd.merge(admission_df_poll.copy(), geog_ccg[['Area Name', 'LAT', 'LONG']], how='inner', on='Area Name') 
# prevalence_df_poll_latlong = pd.merge(prevalence_df_poll.copy(), geog_ccg[['Area Name', 'LAT', 'LONG']], how='inner', on='Area Name')


# # output the data files - to be cleaned up this week... 
# admission_df_poll_latlong.to_csv('../data_output/joined_health_pollution/admission_pollution.csv', encoding='utf-8')
# prevalence_df_poll_latlong.to_csv('../data_output/joined_health_pollution/prevalence_pollution.csv', encoding='utf-8')

# print(admission_df_poll_latlong.shape)
# print(prevalence_df_poll_latlong.shape)

Output melted DF files to CSV (preferred format for front end)

In [98]:
# admission_df_poll.head()

# pd.melt(admission_df_poll_latlong, id_vars=['Year', 'Area Name', 'CCG Location', 'CCG_Poll_Map', 'LAT', 'LONG'],
#         var_name='Indicator Name', value_name='Value').to_csv('../data_output/joined_health_pollution/admission_pollution_melt.csv')


# pd.melt(prevalence_df_poll_latlong, id_vars=['Year', 'Area Name', 'CCG Location', 'CCG_Poll_Map', 'LAT', 'LONG'],
#         var_name='Indicator Name', value_name='Value').to_csv('../data_output/joined_health_pollution/prevalence_pollution_melt.csv')


In [99]:
# dff = pd.melt(prevalence_df_poll_latlong, id_vars=['Year', 'Area Name', 'CCG Location', 'CCG_Poll_Map'],
#         var_name='Indicator Name', value_name='Value')
# dff.head()

In [100]:
# dff[dff['Area Name']=='NHS Oxfordshire CCG' ]

**Conclusions**


**The correlation heatmaps indicate:**

**Disease Prevalence Vs Pollution**
* Distance from the airport is positively corrleated with most disease prevalence indicators), which contradicts our theory that that residents living closer to heathrow experience more health ailments, we would have expected the opposite - further investigation will be required
* PM10 particulate matter is positively correlated with Asthma, Hypertension
* Ozone is positively correlated with COPD prevalence (Further investigation required)


**Hospital Admissions Vs Pollution**
* Various hospital admissions are negatively correlated with heathrow distance, supporting our theory that residents living closer to heathrow experience more health ailments
* Nitrogen Dioxide is positively correlated with 6 hospital admissions
* PM10 particulate matter is positively correlated with with 4 hospital admissions

**Extra**
* Observed pollutants appear to be (weakly) positively correlated with distance, further investigation will be required

**The Multivariate Correlation Plots indicate:**
* There are not many points avaialble for the Sulphur Dioxdide indicator - follow on actions will be given to assess the feasibility of continuing with this indicator. 
* Similarly, action will be undetaken to assess the 2.5OPM indicator (low points)